### Using local mode to run all

In [ ]:
# Ollama RAG Query Notebook


import psycopg2

import ollama
import numpy as np


DB_CONN = "dbname=ragdb user=postgres password=postgres host=localhost"
EMBED_MODEL = "embeddinggemma"
LLM_MODEL = "phi3:mini" #"llama3.1:8b-instruct-q4_K_M"
TOP_K = 3



 Config loaded. Ready to query!


In [ ]:


def embed_text(text: str):
    """Get vector embedding from Ollama embedding model."""
    response = ollama.embeddings(model=EMBED_MODEL, prompt=text)
    return response["embedding"]

def get_pg_connection():
    """Create a PostgreSQL connection."""
    return psycopg2.connect(DB_CONN)

def search_similar(query, top_k=1):
    """Search semantically similar JSON docs for a given query."""
    q_emb = embed_text(query)
    conn = get_pg_connection()
    cur = conn.cursor()

    cur.execute("""
        SELECT filename, content
        FROM json_docs
        ORDER BY embedding <#> %s::vector
        LIMIT %s;
    """, (q_emb, top_k))

    results = cur.fetchall()
    cur.close()
    conn.close()
    return results


def ask_llm(query, context_docs):
    """Ask the LLM using retrieved JSON context."""
    context_text = "\n\n".join([content for _, content in context_docs])
    prompt = f"""
You are a job search assistant. Simply extract and display the job details from the job_information section.

Format EXACTLY like this (no additional text or summaries):

Found matching job:

Position: [title from job_information]
Company: [client from job_information]
Location: [location from job_information]

Would you like to see more details? Reply with 'more'.

Note: Just show exactly what's in the fields. No modifications or summaries. No additional text or explanations.
If no job found, just write: "No matching jobs found."

Context:
{context_text}

Question:
{query}
"""

    stream = ollama.chat(model=LLM_MODEL, messages=[{"role": "user", "content": prompt}], stream=True)
    response = ""
    for chunk in stream:
        if "message" in chunk and "content" in chunk["message"]:
            print(chunk["message"]["content"], end="", flush=True)
            response += chunk["message"]["content"]
    print("\n")
    return response

In [17]:
conn = get_pg_connection()
cur = conn.cursor()
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()
cur.close()
conn.close()
print(" pgvector extension enabled!")


 pgvector extension enabled!


In [18]:
# Load and embed JSON files into PostgreSQL + pgvector

import os
import json
from psycopg2.extras import execute_values

DATA_DIR = os.path.expanduser("~/learn_llm/job_search/mandates/")  # change if needed
def initialize_table():
    """Create table if it doesn’t exist."""
    conn = get_pg_connection()
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS json_docs (
            id SERIAL PRIMARY KEY,
            filename TEXT,
            content TEXT,
            job_information JSONB,
            embedding VECTOR(768)
        );
    """)
    conn.commit()
    cur.close()
    conn.close()
    print(" Table ready: json_docs")

def insert_json_files():
    """Embed each JSON file and store it in the DB."""
    conn = get_pg_connection()
    cur = conn.cursor()

    for filename in os.listdir(DATA_DIR):
        if not filename.endswith(".json"):
            continue

        path = os.path.join(DATA_DIR, filename)
        with open(path, "r") as f:
            job_info = json.load(f)
            content = job_info['job_information']['description']

        # Skip if already inserted
        cur.execute("SELECT COUNT(*) FROM json_docs WHERE filename=%s;", (filename,))
        if cur.fetchone()[0] > 0:
            print(f" Skipping (already embedded): {filename}")
            continue

        print(f" Embedding {filename} ...")
        embedding = embed_text(content)

        execute_values(
            cur,
            "INSERT INTO json_docs (filename, content, embedding, job_information) VALUES %s",
            [(filename, content, embedding, json.dumps(job_info))]
        )
        conn.commit()
        print(f"Inserted {filename}")

    cur.close()
    conn.close()
    print("\n All JSON files processed and stored!")

# --- Run these setup steps ---
initialize_table()
insert_json_files()


 Table ready: json_docs
 Skipping (already embedded): 12_DMART_Cashier.json
 Skipping (already embedded): 09_FOXCONN-Hon-Hai-Technology-India-Mega_Machine-Operator.json
 Skipping (already embedded): 04_SBI-Cards_Sales-Agent.json
 Skipping (already embedded): 10_WISTRON-Infocomm-Manufacturing-Pvt-Ltd_Operator.json
 Skipping (already embedded): 01_RBL-Bank-Limited_Sales-Officer.json
 Skipping (already embedded): 17. Reliance-Retail-Ltd_Customer-Service-Associate.json
 Skipping (already embedded): 08_Samsung-India-Electronics-Pvt-Ltd_Sales-Executive-Consultant.json
 Skipping (already embedded): 15. Phonepe_Business-Development-Executive.json
 Skipping (already embedded): 07_DMART_Picker-Packer.json
 Skipping (already embedded): 13_TEl-Components-Private-Limited_Junior-Technician.json
 Skipping (already embedded): 16. Cash-Management-System_ATM-Cash-Loader.json
 Skipping (already embedded): 14. TATA-Electronics-Private-Limited-MG_Junior-Technician.json
 Skipping (already embedded): 11_Tata

In [ ]:


query = "i need job related picker packer"
print(f"🔍 Searching for: {query}")

docs = search_similar(query)

print(f"Retrieved {len(docs)} relevant documents:\n")
for filename, content in docs:
    print(" -", filename)


Searching for: i need job related picker packer
Retrieved 1 relevant documents:

 - 07_DMART_Picker-Packer.json


In [ ]:


response = ask_llm(query, docs)
print("\nFinal Response:\n", response)


Found matching matching job:

Position job:

Position: Picker Packer
Company: DMART
Location: All India
: Picker Packer
Company: DMART
Location: All India

Would you like
Would you like to see more to see more details? details? Rep Replyly with with ' 'moremore'.'.


Final Response:
 Found matching job:

Position: Picker Packer
Company: DMART
Location: All India

Would you like to see more details? Reply with 'more'.



Final Response:
 Found matching job:

Position: Picker Packer
Company: DMART
Location: All India

Would you like to see more details? Reply with 'more'.
